In [ ]:
from spaces import *
from mark_constants import *
def draw(particles, start, goal, sensor1, sensor2, counter=None):
    space = GridSpace([70*2,30*2])
    normalizer = np.array([7.,3.])
    space.draw_line(start/normalizer, goal/normalizer, LIGHT_RED)
    space.draw_point(start/normalizer, GREEN)
    space.draw_point(goal/normalizer, RED)
    space.draw_point(sensor1/normalizer, BLUE)
    space.draw_point(sensor2/normalizer, LIGHT_BLUE)
    for p in particles:
        space.draw_point(p/normalizer, LIGHT_GREEN)
        
    if counter is None: space.display()
    else: space.display(path='particle_filter/%s.png' % str(counter).zfill(2))

In [ ]:
import numpy as np
from scipy.stats import norm

M = 2000
process_noise_sd = 0.2
sensor_noise_sd = 0.25
start = np.array((1., 1.))
goal = np.array((6., 1.))
sensor1 = np.array((2., 0.))
sensor2 = np.array((5., 0.))


particles = []
for i in range(M):
    p = np.random.rand(2) * np.array([7,3])
    particles.append(p)
particles = np.array(particles)

#true state
state = np.random.rand(2) * np.array([7,3])
belief = particles

def get_distance(p):
    distance1 = np.linalg.norm(p-sensor1)
    distance2 = np.linalg.norm(p-sensor2)
    return min(distance1, distance2)
    

for t in range(15):
    diff = goal - state #follow predefined path
    
    if abs(diff[0]) > abs(diff[1]):
        mov = [1,0] if diff[0] > 0 else [-1,0]
    else:
        mov = [0,1] if diff[1] > 0 else [0,-1]
    mov = np.array(mov)
    noise = np.random.normal(0, process_noise_sd) * mov
    mov = mov*0.2
    
    state += mov + noise
    distance = get_distance(state)
    distance += np.random.normal(0, sensor_noise_sd)
    
    #generate particles using model probability p(x_t|x_t-1, u_t) = N(predicted position, process noise)
    for i in range(M):
        particles[i] = np.random.normal(particles[i]+mov, process_noise_sd, 2)
        
    #redraw from generated particles with weights proportional to sensor model p(z_t|x_t)
    weights = np.zeros(M)
    for i in range(M):
        distance_particle = get_distance(particles[i])
        #p(distance) where p(x)~N(distance_particle, sensor_noise_sd)
        weights[i] = norm.pdf(distance, distance_particle, sensor_noise_sd)
    weights /= np.sum(weights)
    choices = np.random.choice(M, size=M, replace=True, p=weights)
    particles = particles[choices]
    
    draw(particles, start, goal, sensor1, sensor2, t)
        

